In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('./data/Flow.csv')
data.head()

In [ ]:
data = data[(data['LANE_ID'] == 1) & (data['ARM_ID'] == 'E')]
flow = data.loc[:,'VOLUMN_5MIN']
flow.head()

In [ ]:
flow_norm = flow/max(flow)
flow_norm

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig1 = plt.figure(figsize=(100,50))
plt.plot(flow)
plt.title('VOLUMN_5MIN')
plt.xlabel('time')
plt.ylabel('volumn')
plt.show()

In [ ]:
def extract_data(data,time_step):
    X = []
    y = []
    for i in range(len(data)-time_step):
        X.append([a for a in data[i:i+time_step]])
        y.append(data[i+time_step])
    X=np.array(X)
    X = X.reshape(X.shape[0],X.shape[1],1)
    y = np.array(y)
    return X,y

In [ ]:
time_step = 30

In [ ]:
X,y = extract_data(flow_norm,time_step)
print(X.shape,len(y))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN
model = Sequential()
model.add(SimpleRNN(units = 5,input_shape=(time_step,1),activation='relu'))
model.add(Dense(units=1,activation='linear'))
model.compile(optimizer = 'adam',loss = 'mean_squared_error')
model.summary()

In [ ]:
model.fit(X,y,batch_size=100,epochs=200)

In [ ]:
y_train_predict = model.predict(X)*max(flow)
y_train = y*max(flow)
y_train_predict

In [ ]:
fig2 = plt.figure(figsize=(100,50))
plt.plot(y_train)
plt.plot(y_train_predict)
plt.title('VOLUMN_5MIN')
plt.xlabel('time')
plt.ylabel('volumn')
plt.show()

In [ ]:
start_index = len(flow) - time_step  # 开始索引
end_index = start_index + time_step  # 结束索引

# 准备输入数据
X_pred = np.array(flow[start_index:end_index]).reshape(1, time_step, 1)

In [ ]:
# 进行预测
time_length = 24 * 60 // 5  # 接下来要预测的时间长度
y_pred = []
for i in range(time_length):
    pred = model.predict(X_pred)
    y_pred.append(pred[0, 0])
    X_pred[:, :-1, :] = X_pred[:, 1:, :]
    X_pred[:, -1, :] = pred.reshape(1, 1, 1)


In [ ]:
[i*max(flow) for i ]